<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Agent/blob/main/autogen/AutoGen_v0_7_4_MultiAgent_Teams_OpenAI_ko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoGen v0.7.4 — Multi‑Agent Teams (OpenAI 버전 · Colab 실습)

이 노트북은 **AutoGen AgentChat Teams 튜토리얼**을 바탕으로, OpenAI API를 사용해
**멀티에이전트 팀(RoundRobin/Selector)**을 구성·실행·관찰하는 한국어 실습입니다.

## 학습 목표
- Team 개념 이해 및 팀 프리셋(RoundRobin/Selector) 실습
- 종료 조건(termination)과 세션 **reset/resume** 운용
- `Console(...run_stream)`으로 스트리밍 관찰 디버깅


In [1]:
# 0) 설치
!pip -q install "autogen-agentchat" "autogen-ext[openai]" nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 20.3 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade autogen-agentchat

In [10]:
import autogen_agentchat
print(autogen_agentchat.__version__)

0.7.4


## 1. 환경 변수 설정 — OpenAI API 키
- 반드시 `OPENAI_API_KEY`가 있어야 합니다.
- Colab에서는 아래 셀에 직접 키를 넣거나, 런타임 → 환경 변수 메뉴를 사용하세요.

In [2]:
from google.colab import userdata
import openai
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key  = os.environ["OPENAI_API_KEY"]

## 2. 모델 클라이언트 만들기 (OpenAI)
- `OpenAIChatCompletionClient`로 모델 접근
- 기본 모델은 `gpt-4o-mini` (필요시 `gpt-4o` 등으로 변경)


In [3]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",   # gpt-4o, gpt-4.1 등으로 교체 가능
)
print("✅ OpenAI 모델 클라이언트 초기화 완료")

✅ OpenAI 모델 클라이언트 초기화 완료


## 3. RoundRobin 팀 — 생성자(Creator) + 비평가(Critic)
두 명의 AssistantAgent가 **라운드로빈**으로 번갈아 발화합니다.
- Creator: 해결안을 단계별로 작성(한국어)
- Critic: 정확성·명료성 검토 → **만족 시 'APPROVE' 포함**
- `TextMentionTermination("APPROVE")`로 종료 조건 설정
- `Console(...run_stream)`으로 스트리밍 관찰


In [12]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-2024-08-06",
    # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

In [13]:
# Use `asyncio.run(...)` when running in a script.
result = await team.run(task="가을에 대한 짧은 시를 써 보세요.") # Write a short poem about the fall season.
print(result)

messages=[TextMessage(id='9fd1b34b-aa9d-412f-b104-606ea12d89d6', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 8, 0, 7, 0, 749891, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(id='0bd9439f-e8c8-4507-b3a3-03ccbed18904', source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=110), metadata={}, created_at=datetime.datetime(2025, 9, 8, 0, 7, 4, 310250, tzinfo=datetime.timezone.utc), content="Golden leaves cascade from trees,  \nWhispering tales in the crisp breeze.  \nPumpkin scents and cider's bite,  \nCloak the earth in cozy light.  \n\nSweaters wrap against the chill,  \nAs twilight paints the world so still.  \nSmoky skies and early dusk,  \nNature's sigh as summer's husk.  \n\nFields of amber, skies of slate,  \nAutumn's dance—a fleeting fate.  \nA tapestry of red and gold,  \nIn fall's embrace, the heart consoled.  ", type='TextMessage'), TextMessage

In [15]:
# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="가을에 대한 짧은 시를 써 보세요."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

id='5fbe3bfa-170f-4c7c-903d-3139343a31b9' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 9, 8, 0, 9, 32, 561852, tzinfo=datetime.timezone.utc) content='가을에 대한 짧은 시를 써 보세요.' type='TextMessage'
id='20af138f-35d8-4ba6-9952-820139c9da61' source='primary' models_usage=RequestUsage(prompt_tokens=33, completion_tokens=55) metadata={} created_at=datetime.datetime(2025, 9, 8, 0, 9, 34, 228546, tzinfo=datetime.timezone.utc) content='붉은 잎이 춤추는 길,  \n서늘한 바람 속에 속삭임이 있어,  \n낙엽은 바닥에 깔린 곡선,  \n가을은 조용히 이야기를 전하네.' type='TextMessage'
id='6322b66a-5cb0-4b6e-8f92-6b0fafeb4cca' source='critic' models_usage=RequestUsage(prompt_tokens=106, completion_tokens=178) metadata={} created_at=datetime.datetime(2025, 9, 8, 0, 9, 38, 625640, tzinfo=datetime.timezone.utc) content='시의 분위기와 계절의 특징을 잘 표현하셨습니다. 가을의 감성과 자연의 변화를 느낄 수 있습니다. 하지만 \'조용히 이야기를 전하네\'라는 표현이 다소 추상적으로 느껴질 수 있습니다. \'이야기\' 대신 좀 더 구체적인 이미지를 사용하면 시각적 상상이 더 풍부해질 것 같습니다. 예를 들어 가을이 들려주는 이야기가 무엇인지 간단한 한 문장을 추가해보는 건 어떨까요? \n\n예시:

## 4. 팀 상태 관리 — reset / resume
- **reset()**: 이전 작업과 무관한 새 작업 전, 팀 상태 초기화
- **resume**: 같은 팀으로 `run_stream()`을 **다시 호출**하면 직전 이후 발화자부터 이어감
- 아래 셀은 RoundRobin 팀으로 **두 작업을 연속 실행**해봅니다.

In [16]:
# Create a new team with an external termination condition.
external_termination = ExternalTermination()
team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition=external_termination | text_termination,  # Use the bitwise OR operator to combine conditions.
)

# Run the team in a background task.
run = asyncio.create_task(Console(team.run_stream(task="Write a short poem about the fall season.")))

# Wait for some time.
await asyncio.sleep(0.1)

# Stop the team.
external_termination.set()

# Wait for the team to finish.
await run

---------- TextMessage (user) ----------
Write a short poem about the fall season.
---------- TextMessage (primary) ----------
Golden leaves in crisp air dance,  
Whispering secrets at a glance.  
Under the trees, a carpet of gold,  
Stories of autumn quietly unfold.


TaskResult(messages=[TextMessage(id='174a32be-b719-45d4-9a73-636fe10b6686', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 8, 0, 11, 14, 155549, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(id='aac2ea47-da13-4f48-8c97-198b485099e3', source='primary', models_usage=RequestUsage(prompt_tokens=411, completion_tokens=33), metadata={}, created_at=datetime.datetime(2025, 9, 8, 0, 11, 15, 654778, tzinfo=datetime.timezone.utc), content='Golden leaves in crisp air dance,  \nWhispering secrets at a glance.  \nUnder the trees, a carpet of gold,  \nStories of autumn quietly unfold.', type='TextMessage')], stop_reason='External termination requested')

## 5. Selector 팀 — 발화자 선택
여러 전문가 에이전트 중 **LLM이 다음 발화자**를 고르는 패턴입니다.
- Planner: 계획 수립(번호 목록)
- Researcher: 웹·지식 요약(불릿)
- Coder: 파이썬 코드만 출력(필요 시)
- 사용 예: 복합 문제에서 **누가 말해야 하는지**를 LLM이 선택하도록 함

In [ ]:
from autogen_agentchat.teams import SelectorGroupChat

planner = AssistantAgent(
    name="planner",
    system_message="한국어로 단계별 계획을 번호 목록으로 제시하세요.",
    model_client=model_client,
    model_client_stream=True,
)
researcher = AssistantAgent(
    name="researcher",
    system_message="핵심 지식을 한국어 불릿 포인트로 요약하세요. (가정/한계 명시)",
    model_client=model_client,
    model_client_stream=True,
)
coder = AssistantAgent(
    name="coder",
    system_message="파이썬 코드만 출력하세요. 불필요한 설명은 하지 마세요.",
    model_client=model_client,
    model_client_stream=True,
)

team_sel = SelectorGroupChat([planner, researcher, coder], model_client=model_client) # Added model_client

async def demo_selector():
    task = (
        "웹에서 파이썬 웹스크래핑 학습 경로를 요약하고, 예제 코드(BeautifulSoup)를 간단히 제시.\n"
        "가정과 한계도 명시."
    )
    await Console(team_sel.run_stream(task=task))

await demo_selector()

## 6. 실습 과제
1) **종료 조건 변경**: `TextMentionTermination("DONE")`으로 바꿔 동작 비교
2) **전문가 추가**: Selector 팀에 "테스터" 에이전트를 추가하여 코드 검증 규칙을 부여
3) **다중 도구 연계**: 특정 에이전트에 간단한 도구(예: 계산기 함수)를 연결해 함수 호출 흐름 관찰
4) **팀 재사용**: 같은 팀 인스턴스를 재사용하여 **여러 작업**을 연속 실행하고 reset/resume 패턴 숙달


## 7. 참고
- AgentChat Teams 튜토리얼: MagenticOne/Swarm 등 고급 팀 패턴
- 스트리밍 디버깅은 실전에서 **필수** — 함수 호출/반환, 종료 타이밍을 확인하세요.
- 무한 루프 방지: **종료 조건 + 최대 턴** 정책을 병행하세요.
